In [ ]:
import ot
import numpy as np
import cv2
import time
import matplotlib.pyplot as plt
from PIL import Image
import torch


In [ ]:
### GOOD CPU between 2 images

def load_and_prep_image(path, size=(80, 80)):
    img = Image.open(path)
    
    if img.mode in ('RGBA', 'LA', 'P'):
        background = Image.new('RGB', img.size, (255, 255, 255)) # remove the transparent background if it's there
        if img.mode == 'P':
            img = img.convert('RGBA')
        background.paste(img, mask=img.split()[-1])
        img = background
    
    img = img.convert('L')
    img = img.resize(size)
    img_array = np.array(img, dtype=np.float64)
    
    img_array = img_array.max() - img_array
    
    threshold = 0.02 * img_array.max()
    img_array[img_array < threshold] = 0
    
    # Add epsilon
    img_array += 1e-6
    
    return img_array


def ot_interpolate(img1, img2, t, reg=0.004):
    size = img1.shape[0]
    
    d1 = img1 / np.sum(img1)
    d2 = img2 / np.sum(img2)
    
    d1_flat = d1.flatten()
    d2_flat = d2.flatten()
    
    A = np.vstack((d1_flat, d2_flat)).T
    
    x = np.arange(size, dtype=np.float64)
    y = np.arange(size, dtype=np.float64)
    X, Y = np.meshgrid(x, y)
    coords = np.stack((X.flatten(), Y.flatten()), axis=1)
    
    M = ot.dist(coords, coords, metric='sqeuclidean')
    M = M / M.max()
    
    weights = np.array([1 - t, t])
    barycenter_flat = ot.bregman.barycenter(A, M, reg, weights)
    
    result = barycenter_flat.reshape((size, size))
    
    return result


def save_ot_image(img_array, output_path):
    if img_array.max() > 0:
        img_array = img_array / img_array.max()
    
    img_array = 1.0 - img_array

    if img_array.max() > img_array.min():
        img_normalized = (img_array - img_array.min()) / (img_array.max() - img_array.min())
        img_normalized = (img_normalized * 255).astype(np.uint8)
    else:
        img_normalized = np.zeros_like(img_array, dtype=np.uint8)
    
    img_pil = Image.fromarray(img_normalized, mode='L')
    img_pil.save(output_path)


img1 = load_and_prep_image("walking/frame0023.png", size=(160, 120))
img2 = load_and_prep_image("jogging/frame0029.png", size=(160, 120))

t = 0
i = 1
while t < 1:
    result = ot_interpolate(img1, img2, t=t, reg=0.0001)
    save_ot_image(result, f"output/frame{i:04}.png")
    t += .05
    i +=1

In [ ]:
### GOOD GPU

import torch
import ot
import numpy as np
from PIL import Image
import os
import glob

WIDTH = 160
HEIGHT = 120
REG_VAL = 1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on: {device}")

def load_image_gpu(path, size=(WIDTH, HEIGHT)):
    img = Image.open(path).convert('RGBA')
    
    bg = Image.new('RGBA', img.size, (255, 255, 255))
    img = Image.alpha_composite(bg, img).convert('L')
    img = img.resize(size)
    
    img_array = np.array(img, dtype=np.float64)
    t_img = torch.tensor(img_array, dtype=torch.float64, device=device)
    
    t_img = t_img.max() - t_img
        
    t_img[t_img < 0.05 * t_img.max()] = 0
    
    t_img = t_img / torch.sum(t_img)
    
    return t_img.flatten()

def get_cost_matrix_gpu(width, height):
    x = torch.arange(width, dtype=torch.float64, device=device)
    y = torch.arange(height, dtype=torch.float64, device=device)
    
    Y, X = torch.meshgrid(y, x, indexing='ij') 
    
    coords = torch.stack((X.flatten(), Y.flatten()), dim=1)
    
    M = torch.cdist(coords, coords, p=2) ** 2
    M = M / M.max()
    return M

def save_ot_image_gpu(tensor, output_path, size=(WIDTH, HEIGHT)):
    arr = tensor.cpu().numpy().reshape(size[1], size[0]) # reshape(Height, Width)
    
    if arr.max() > 0:
        arr = arr / arr.max()
        
    arr = 1.0 - arr
    
    arr = (arr * 255).clip(0, 255).astype(np.uint8)
    
    Image.fromarray(arr).save(output_path)

M_gpu = get_cost_matrix_gpu(WIDTH, HEIGHT)

os.makedirs("output_gpu", exist_ok=True)

start_frame = 30
t = .05

for i in range(1, 21):
    
    t = .05 * i
    
    path1 = f"walking/frame{i:04d}.png"
    path2 = f"jogging/frame{i:04d}.png"
    
    d1 = load_image_gpu(path1, size=(WIDTH, HEIGHT))
    d2 = load_image_gpu(path2, size=(WIDTH, HEIGHT))
    
    A_gpu = torch.stack((d1, d2), dim=1)

    weights = torch.tensor([1 - t, t], dtype=torch.float64, device=device)
    barycenter = ot.bregman.barycenter(A_gpu, M_gpu, REG_VAL, weights)
    
    save_ot_image_gpu(barycenter, f"output_gpu/out_{i:04d}.png", size=(WIDTH, HEIGHT))

In [ ]:
# CONV 2D from scratch

import numpy as np
from scipy.ndimage import gaussian_filter
from PIL import Image
import os

class ConvolutionalBarycenter:
    def __init__(self, size, sigma=2.0, iterations=15):
        """
        size: (Height, Width)
        sigma: The 'blur' radius. Higher = smoother/more stable. Lower = sharper/unstable.
               sigma=2.0 is a good starting point for 160x120 images.
        iterations: 10-20 is usually enough.
        """
        self.height, self.width = size
        self.sigma = sigma
        self.iters = iterations
        self.epsilon = 1e-10 # Tiny number to prevent division by zero

    def solve(self, images, weights):
        """
        images: List of 2D numpy arrays (normalized).
        weights: List of floats (sum to 1.0).
        """
        v = np.ones((len(images), self.height, self.width), dtype=np.float64)
        
        barycenter = None
        
        for i in range(self.iters):
            barycenter_accum = np.ones((self.height, self.width), dtype=np.float64)
            
            for k in range(len(images)):
                Kv = gaussian_filter(v[k], sigma=self.sigma, mode='constant', cval=0.0)
                
                barycenter_accum *= (Kv + self.epsilon) ** weights[k]

            barycenter = barycenter_accum

            for k in range(len(images)):
                Kb = gaussian_filter(barycenter, sigma=self.sigma, mode='constant', cval=0.0)
                v[k] = images[k] / (Kb + self.epsilon)

        return barycenter

def load_image(path, size):
    img = Image.open(path).convert("RGBA")
    bg = Image.new("RGBA", img.size, (255, 255, 255, 255))
    img = Image.alpha_composite(bg, img).convert("L")
    img = img.resize((size[1], size[0]))
    
    arr = np.array(img, dtype=np.float64)

    if arr.mean() > 127:
        arr = arr.max() - arr
    
    arr = gaussian_filter(arr, sigma=1.0) 

    arr[arr < 0.05 * arr.max()] = 0

    arr += 1e-9
    arr /= arr.sum()
    return arr

def save_image(arr, path):
    if arr.max() > 0: arr /= arr.max()
    arr = 1.0 - arr
    img = Image.fromarray((arr * 255).astype(np.uint8))
    img.save(path)

H, W = 120, 160
SIGMA = .8
solver = ConvolutionalBarycenter(size=(H, W), sigma=SIGMA, iterations=100)

os.makedirs("output_custom", exist_ok=True)

print("Starting Custom Solver...")
frame_count = 30
t = 0.0

while t <= 1.0:
    path1 = f"walking/frame{frame_count:04d}.png"
    path2 = f"jogging/frame{frame_count:04d}.png"
    
    if not os.path.exists(path1): break

    img1 = load_image(path1, (H, W))
    img2 = load_image(path2, (H, W))
    
    result = solver.solve([img1, img2], weights=[1-t, t])
    
    save_image(result, f"output_custom/frame{frame_count:04d}.png")
    print(f"Frame {frame_count} (t={t:.2f})")
    
    t += 0.05
    frame_count += 1

print("Done.")

In [ ]:
# LINEAR

import numpy as np
from PIL import Image
import os

SIZE = (160, 120)

def load_image_linear(path):
    try:
        img = Image.open(path).convert("RGBA")
        
        bg = Image.new("RGBA", img.size, (255, 255, 255, 255))
        img = Image.alpha_composite(bg, img).convert("L")
        img = img.resize(SIZE)
        
        arr = np.array(img, dtype=np.float64) / 255.0
        return arr
    except Exception:
        return None

def save_frame(arr, path):
    arr = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
    Image.fromarray(arr).save(path)

os.makedirs("output_linear", exist_ok=True)
print("Starting Linear Cross-Dissolve...")

t = 0
for i in range(30, 50):
    path1 = f"walking/frame{i:04d}.png"
    path2 = f"jogging/frame{i:04d}.png"
    
    if not os.path.exists(path1): break
    
    img1 = load_image_linear(path1)
    img2 = load_image_linear(path2)
    
    t += .05
    
    result = (1 - t) * img1 + t * img2
    
    save_frame(result, f"output_linear/frame{i:04d}.png")
    
    if i % 5 == 0: print(f"Frame {i} done")

print("Done. Check 'output_linear' folder.")

